In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from tqdm import tqdm
from pathlib import Path

WORK_DIR = Path(Path.cwd()).parent
sys.path.append(str(WORK_DIR))
from src.datasets import get_dataset, get_dataloader
from src.utils import parse_data_cfg, IMG, VOC, DATA_DIR, YOLO

In [ ]:
cfg = parse_data_cfg(WORK_DIR/'data_cfg'/'yolov2_voc_data.cfg')
epoch = 300
exp_dir = cfg["exp_dir"]
data_split = 'test'
split_set = data_split + '_set'

# Visualize Data Loader

In [ ]:
dataset_kwargs = {'split_set': cfg[split_set]}
cfg['aug'] = None
dataset   = get_dataset(cfg, dataset_kwargs)
sampler   = None
shuffle   = cfg['shuffle']
kwargs = {'batch_size'  :   int(cfg['batch_size']),
          'shuffle'     :   shuffle,
          'num_workers' :   int(cfg['num_workers']),
          'pin_memory'  :   True}
data_loader = get_dataloader(dataset, sampler, kwargs)

In [ ]:
idx = 0 
for i, (img, labels, imgpath) in enumerate(data_loader):
    if i == idx:
        batch_size = img.shape[0]
        img = img.cpu().numpy()
        img = np.swapaxes(img, 2, 3)
        img = np.swapaxes(img, 1, 3)
        img = IMG.scale_img_255(img)
        labels = labels.cpu().numpy()
        break
    i += 1

In [ ]:
new_labels = []
for l in labels:
    new_labels.append(VOC.filter_zero_from_labels(l))

fig, ax = plt.subplots(4, 4, figsize=(15, 15))
idx = 0
for i in range(4):
    for j in range(4):
        idx = np.ravel_multi_index((i, j), (5, 5))
        if idx >= len(new_labels):
            break              
        cur_img = img[idx]
        boxes = new_labels[idx][:, 1:5]     
        ax[i, j].imshow(cur_img)
        VOC.draw_bbox(ax[i, j], boxes, (cur_img.shape[1], cur_img.shape[0]), class_id=new_labels[idx][:, 0])

# Evaluation

In [ ]:
filename = Path(DATA_DIR)/exp_dir/'predict_{}_{}_'.format(epoch, data_split)
year = '2007'
classes = VOC.CLASS_LIST

img_root = Path(DATA_DIR)/cfg[split_set]
with open(img_root, 'r') as file:
    lines = file.readlines()    

In [ ]:
voc_dir = Path(DATA_DIR)/'VOCdevkit'
VOC.do_python_voc_eval(str(filename), voc_dir, classes, year, data_split)

# Visualize Prediction

In [ ]:
idx = 4
imgpath = Path(DATA_DIR)/'VOCdevkit'/'VOC2007'/'JPEGImages'/'{:06d}.jpg'.format(idx)
labpath = Path(DATA_DIR)/'VOCdevkit'/'VOC2007'/'labels'/'{:06d}.txt'.format(idx)
img = Image.open(imgpath)
w, h = img.size
img = img.resize((416, 416))
img = np.asarray(img)

# Predicted
filename = Path(DATA_DIR)/exp_dir/'predict_{}_{}_all.txt'.format(epoch, data_split)
with open(filename, 'r') as f:
    lines = f.readlines()    
lines = np.asarray([i.strip().split() for i in lines]).astype('float32')
lines[:, 0] = [int(i) for i in lines[:, 0]]
chosen_lines = lines[np.where(lines[:, 0] == idx)]
xyxy = chosen_lines[:, 2:6]
boxes = YOLO.xyxy2xywh(xyxy)
boxes[:, 0] /= w
boxes[:, 1] /= h
boxes[:, 2] /= w
boxes[:, 3] /= h
# Ground truth
with open(labpath, 'r') as f:
    labels_gt = f.readlines()
labels_gt = np.asarray([i.strip().split() for i in labels_gt]).astype('float32')
boxes_gt = labels_gt[:, 1:5]

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(img)
VOC.draw_bbox(ax, boxes_gt, (img.shape[1], img.shape[0]), class_id=labels_gt[:, 0], color='b')
VOC.draw_bbox(ax, boxes, (img.shape[1], img.shape[0]), class_id=chosen_lines[:, -1], color='r')
plt.show()